In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
with open('/content/drive/My Drive/House MD Text Generation/Final Merged UTF8 2.txt','r',encoding = "utf8") as f:
  text = f.read().lower().replace('\n',' \n ')
print(text[1:1000])

In [ ]:
words = [w for w in text.split(' ') if w != '']

In [ ]:
words[1:10]

In [ ]:
len(words)

In [ ]:
word_frequencies = {}
#i = 1
for word in words:
    #print(i)
    word_frequencies[word] = word_frequencies.get(word, 0) + 1
    #i = i + 1
#word_frequencies


In [ ]:
min_frequency = 10
ignored_words = set()
for k,v in word_frequencies.items():
  if word_frequencies[k] < min_frequency:
    ignored_words.add(k)


In [ ]:
len(ignored_words)

In [ ]:
def remove_values_from_list(the_list, ignored_words):
   return [value for value in the_list if value not in ignored_words]

In [ ]:
words = remove_values_from_list(words,ignored_words)


In [ ]:
len(words)

In [ ]:
len(set(words))

In [ ]:
print(words[1:10])

In [ ]:
text_edited = ' '.join(words)
with open("/content/drive/My Drive/House MD Text Generation/House Edited Frequency.txt",'w') as file:
  file.write(text_edited)

In [ ]:
word_indices = dict((word, index) for index, word in enumerate(set(words)))
indices_word = dict((index, word) for index, word in enumerate(set(words)))

In [ ]:
vectorized_words = []
for word in words:
    vectorized_words.append(word_indices[word])


In [ ]:
vectorized_words = np.array(vectorized_words)
#vectorized_ignored_words = np.array(vectorized_ignored_words)

In [ ]:
vectorized_words.shape

In [ ]:
vectorized_words

In [ ]:
vectorized_words = np.reshape(vectorized_words,(-1,1))
vectorized_words

In [ ]:
vectorized_words.shape

In [ ]:
'''from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
vectorized_words = scaler.fit_transform(vectorized_words)
vectorized_words'''

In [ ]:
X_train = []
y_train = []
seq_length = 500
for i in range(seq_length,len(vectorized_words)):
  X_train.append(vectorized_words[i-seq_length:i,0])
  y_train.append(vectorized_words[i,0])

In [ ]:
#X_train , y_train = np.array(X_train), np.array(y_train)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(len(set(words))))
model.add(keras.layers.Activation('softmax'))

In [ ]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, seq_length, len(set(words))), dtype=np.float16)
        #print(x.shape)
        y = np.zeros((batch_size,len(set(words))) , dtype=np.float16)
        for i in range(batch_size):
          for t, num in enumerate(sentence_list[index]):
            x[i, t, num] = 1
          y[i, next_word_list[i]] = 1

          index = index + 1
          if index == len(sentence_list):
              index = 0
        yield x, y

In [ ]:
#X_train.shape

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
batch_size = 75

In [ ]:
model.fit(generator(X_train,y_train,batch_size),steps_per_epoch=int(len(X_train)/batch_size)+1,epochs = 1)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
start_index = random.randint(0, len(text_edited) - seq_length - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text_edited[start_index : start_index + seq_length]
    print('...Generating with seed: "' + sentence + '"')
    gen_length = 1000
    for i in range(gen_length):
        x_pred = np.zeros((1, seq_length, len(set(words))))
        for t, num in enumerate(sentence):
            x_pred[0, t, num] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]
        sentence = sentence.split(" ")[1:].append(next_word)
        generated += next_word

    print("...Generated: ", generated)
    print()